In [10]:
%matplotlib tk

from hmmlearn import hmm
import numpy as np
import matplotlib.pyplot as plt


def boundaries(v):
    bounds = [0] + list(np.where(v[:-1] != v[1:])[0] + 1) + [len(v)-1]
    return [(bounds[i], bounds[i+1], v[bounds[i]]) for i in range(len(bounds)-1)]

def to_range(l, cmap):
    total = cmap.N
    idx = int((l[-1] / N) * total)
    return cmap(idx)

FILE = 'data/pills-s0.npy'

# PARAMETERS
N = 5
covariance_type = 'full'
iters = 100


# TRAIN MODEL
X = np.load(FILE)
model = hmm.GaussianHMM(n_components=N, covariance_type=covariance_type, n_iter=iters)
# train
print('TRAINING MODEL: N={}, covar={}, iters={}'.format(N, covariance_type, iters))
model.fit(X)
# retrieve hidden states
print('DONE TRAINING, PREDICTING OUTPUT')
Z = model.predict(X)
Zb = boundaries(Z)
print('DONE PREDICTING:', len(Zb))
print(Zb)

# PLOTTING
fig, ax = plt.subplots()
# plot signals
ax.plot(X[:,:-3], alpha=.7)
ax.set_ylabel('gyroscope')
ax.set_xlabel('time (milliseconds)')
ax.set_ylim(-5000, 5000)
# plot labels
_cmap = plt.get_cmap('jet')
plots = {}
for l in Zb:
    if l[-1] > 0:
        ev = Zb[l[-1]]
        clr = _cmap(l[-1])
        i = plt.axvspan(l[0], l[1], color=to_range(l, _cmap), alpha=0.5, label=ev)
        plots[ev] = i
# draw
plt.show()

TRAINING MODEL: N=5, covar=full, iters=100


/usr/local/lib/python3.5/dist-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/usr/local/lib/python3.5/dist-packages/hmmlearn/base.py:460: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/usr/local/lib/python3.5/dist-packages/hmmlearn/base.py:468: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/usr/local/lib/python3.5/dist-packages/hmmlearn/base.py:469: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),
/usr/local/lib/python3.5/dist-packages/hmmlearn/base.py:624: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_),


DONE TRAINING, PREDICTING OUTPUT
DONE PREDICTING: 169
[(0, 281, 3), (281, 338, 2), (338, 344, 4), (344, 346, 2), (346, 348, 4), (348, 351, 0), (351, 353, 4), (353, 421, 0), (421, 422, 4), (422, 423, 0), (423, 424, 4), (424, 426, 0), (426, 431, 4), (431, 443, 1), (443, 469, 2), (469, 484, 4), (484, 485, 2), (485, 487, 4), (487, 493, 1), (493, 500, 4), (500, 529, 1), (529, 552, 4), (552, 559, 2), (559, 573, 4), (573, 576, 2), (576, 608, 4), (608, 630, 2), (630, 632, 4), (632, 634, 1), (634, 636, 2), (636, 640, 4), (640, 642, 0), (642, 644, 4), (644, 829, 0), (829, 830, 4), (830, 843, 1), (843, 852, 2), (852, 855, 4), (855, 860, 2), (860, 865, 4), (865, 874, 2), (874, 886, 4), (886, 887, 1), (887, 899, 4), (899, 907, 1), (907, 909, 4), (909, 950, 1), (950, 980, 4), (980, 984, 2), (984, 993, 4), (993, 996, 1), (996, 999, 4), (999, 1007, 2), (1007, 1011, 4), (1011, 1013, 2), (1013, 1030, 4), (1030, 1055, 2), (1055, 1058, 4), (1058, 1295, 0), (1295, 1296, 4), (1296, 1345, 0), (1345, 1349, 1)

/usr/local/lib/python3.5/dist-packages/hmmlearn/base.py:451: RuntimeWarning: divide by zero encountered in log
  n_samples, n_components, np.log(self.startprob_),
/usr/local/lib/python3.5/dist-packages/hmmlearn/base.py:452: RuntimeWarning: divide by zero encountered in log
  np.log(self.transmat_), framelogprob)
